Idea!
Icons zijn de retailers
Orderd by producten die ze verkopen (maybe productcategoriesn getallen toeweizen?)
Groepen gemaakt op Retailer categorie

Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import pyodbc
import sqlite3
import random
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.discriminant_analysis import StandardScaler



Gather data

In [ ]:
def getDbAsDF(conn: sqlite3.Connection, sql: str):
    cursor = conn.cursor()
    cursor.execute(sql)
    data = cursor.fetchall()

    columns = [column[0] for column in cursor.description]

    formatted_data = {}

    for i in range(len(columns)):
        dataList = []
        for j in data:
            dataList.append(j[i])
        formatted_data[columns[i]] = dataList

    df = pd.DataFrame(data=formatted_data, columns=columns)
    return df


# Connect to SQLite database
connection = sqlite3.connect(r'..\..\Data\merged.sqlite')

SQL = r"""
SELECT *
FROM order_header as OH
INNER JOIN retailer as RT ON OH.RETAILER_NAME = RT.COMPANY_NAME
INNER JOIN sales_demographic as SD on RT.RETAILER_CODEMR = SD.RETAILER_CODEMR
INNER JOIN order_details as OD ON OH.ORDER_NUMBER = OD.ORDER_NUMBER
INNER JOIN product as P ON P.PRODUCT_NUMBER = OD.PRODUCT_NUMBER
INNER JOIN product_type as PT ON PT.PRODUCT_TYPE_CODE = P.PRODUCT_TYPE_CODE
"""

DF = getDbAsDF(connection, SQL)
print(DF.columns)

DF[["UNIT_PRICE"]] = DF[["UNIT_PRICE"]].astype(float)
DF[["QUANTITY"]] = DF[["QUANTITY"]].astype(int)
DF[["PRODUCTION_COST"]] = DF[["PRODUCTION_COST"]].astype(float)
DF

Selecting Data

In [ ]:
Retailers = DF["RETAILER_CODE"].unique()
dataset = pd.DataFrame(columns=["Quantity","Totaalprijs","Retailer"])
for number in Retailers:
    quantity = 0
    price = 0
    list1 = []
    list2 = []
    list3 = []
    FoundRows = DF.loc[DF['RETAILER_CODE'] == number]
    #Yes this works, no I cannot wrap my head arround dataframes.
    for row in FoundRows["QUANTITY"]:
        quantity += row
        list1.append(row)
    for row in FoundRows["UNIT_PRICE"]:
        list2.append(row)
    for row in FoundRows["PRODUCTION_COST"]:
        list3.append(row)
    for i in range(len(list1)):
        price+=list1[i]*(list2[i]-list3[i])

    dataset.loc[len(dataset)] = {"Retailer":number,"Quantity":quantity,"Totaalprijs":price}

print(dataset.sort_values(by=['Totaalprijs'], ascending=False))


# Data = DF[["RETAILER_CODE","UNIT_PRICE","QUANTITY","AGE_GROUP_CODE"]]

# Data = Data.T.drop_duplicates().T
# Data.dropna(inplace = True)

scaler = StandardScaler()
scaled_features = scaler.fit_transform(dataset)
scaled_features

Clustering

In [ ]:

kmeans = KMeans(n_clusters=6, init='random',
        n_init=10, max_iter=300,
        tol=1e-04, random_state=0)

y_km = kmeans.fit(dataset)
cluster_labels = kmeans.labels_

plt.scatter(scaled_features[:,0], scaled_features[:,1], c=cluster_labels)
plt.xlabel('Quantity')
plt.ylabel('Price')
plt.title('Clustering Analysis')
plt.grid
plt.legend()
plt.show()

Calculate distortion

In [ ]:
# calculate distortion for a range of number of cluster
distortions = []
for i in range(1, 11):
    km = KMeans(
        n_clusters=i, init='random',
        n_init=10, max_iter=300,
        tol=1e-04, random_state=0
    )
    km.fit(scaled_features)
    distortions.append(km.inertia_)

# plot
plt.plot(range(1, 11), distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()